In [ ]:
#Import the Required Libraries

import pandas as pd
from PIL import Image
import numpy as np

In [ ]:
#Read the Datasets

gallery_df = pd.read_csv('gallery.csv')
queries_df = pd.read_csv('queries.csv')

In [ ]:
!ls

gallery.csv  gallery.zip  queries.csv  queries.zip  sample_data


In [ ]:
!unzip gallery.zip

Archive:  gallery.zip
  inflating: gallery/abiding-debonair-viper-of-downpour.jpg  
  inflating: gallery/able-smoky-echidna-of-discourse.jpg  
  inflating: gallery/aboriginal-maroon-flamingo-of-criticism.jpg  
  inflating: gallery/aboriginal-organic-sturgeon-of-fury.jpg  
  inflating: gallery/aboriginal-rational-kiwi-of-priority.jpg  
  inflating: gallery/abstract-maize-cuscus-of-growth.jpg  
  inflating: gallery/accelerated-enthusiastic-pegasus-of-attraction.jpg  
  inflating: gallery/accelerated-merciful-mongrel-of-exercise.jpg  
  inflating: gallery/accomplished-meaty-ladybug-from-hell.jpg  
  inflating: gallery/accomplished-strict-copperhead-from-mars.jpg  
  inflating: gallery/accurate-hypersonic-clam-of-philosophy.jpg  
  inflating: gallery/acrid-belligerent-mongrel-of-fantasy.jpg  
  inflating: gallery/acrid-horned-peacock-of-current.jpg  
  inflating: gallery/acrid-myrtle-tuatara-of-joy.jpg  
  inflating: gallery/acrid-optimal-mule-of-unity.jpg  
  inflating: gallery/acrid-prop

In [8]:
!unzip queries.zip

Archive:  queries.zip
  inflating: queries/abiding-inchworm-of-ultimate-freedom.jpeg  
  inflating: queries/abiding-industrious-bullfinch-from-heaven.jpeg  
  inflating: queries/abiding-tomato-oarfish-of-excellence.jpg  
  inflating: queries/abiding-warm-buffalo-of-vitality.jpeg  
  inflating: queries/abiding-warping-gibbon-of-acceptance.jpeg  
  inflating: queries/able-cherubic-zebu-from-ganymede.jpeg  
  inflating: queries/able-fervent-pheasant-of-feminism.jpeg  
  inflating: queries/able-fluffy-labrador-of-philosophy.jpeg  
  inflating: queries/able-fuzzy-ladybug-of-tempest.jpeg  
  inflating: queries/able-natural-boa-of-force.jpeg  
  inflating: queries/able-talented-cuttlefish-of-cookies.jpeg  
  inflating: queries/able-tasteful-dalmatian-of-glory.jpeg  
  inflating: queries/aboriginal-purple-viper-of-tempering.jpeg  
  inflating: queries/aboriginal-sparkling-magpie-of-agility.jpg  
  inflating: queries/abstract-radical-lorikeet-of-intensity.jpeg  
  inflating: queries/accelerated

In [ ]:
#Pre-process the Gallery Images - Resize the image to 224x224, Convert the image to a numpy array, Normalize pixel values to [0, 1]

gallery_images = []
for img_path in gallery_df['img_path']:
    img = Image.open(img_path)
    img = img.resize((224, 224)) 
    img = np.array(img) 
    img = img / 255.0 
    gallery_images.append(img)
gallery_images = np.array(gallery_images)

In [9]:
# Preprocess the query images - Resize the image to 224x224, Convert the image to a numpy array, Normalize pixel values to [0, 1]
query_images = []
for i, row in queries_df.iterrows():
    img = Image.open(row['img_path'])
    img = img.resize((224, 224)) 
    img = np.array(img) 
    img = img / 255.0 
    query_images.append(img)
query_images = np.array(query_images)

In [10]:
#Extract the Bounding Boxes from the Queries Dataframe
query_boxes = queries_df[['bbox_x', 'bbox_y', 'bbox_w', 'bbox_h']].values

In [11]:
#Extract Features Based on Pre-trained ResNet50 Model [ImageNet Database]

import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

gallery_features = model.predict(preprocess_input(gallery_images))

query_features = model.predict(preprocess_input(query_images))

61/61 [==============================] - 365s 6s/step


In [12]:
#Compute Cosine Similarities between the Query Features and the Gallery Features
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity(query_features, gallery_features)

In [13]:
#Rank the Gallery Images based on Similarity Scores

sorted_indices = np.argsort(similarities, axis=1)[:, ::-1]
ranked_results = sorted_indices[:, :1000]

In [14]:
#Submission.csv

submission = np.zeros((len(queries_df), 1000))
for i, indices in enumerate(ranked_results):
    submission[i, :] = gallery_df.iloc[indices]['seller_img_id'].values

np.savetxt('submission.csv', submission, delimiter=',')